In [ ]:
"""
.. _ex-source-space-custom-atlas:

=========================================
Source reconstruction with a custom atlas
=========================================

This example shows how to use a custom atlas when performing source reconstruction.
We showcase on the sample dataset how to apply the Yeo atlas during source reconstruction.
You should replace the atlas with your own atlas and your own subject.

Any atlas can be used instead of Yeo, provided each region contains a single label (ie: no probabilistic atlas). 

.. warning:: This tutorial uses FSL and FreeSurfer to perform MRI coregistrations. If you use a different software, replace the coregistration function appropriately.
"""

# Authors: Fabrice Guibert <fabrice.guibert.96@gmail.com>
#
# License: BSD-3-Clause
# Copyright the MNE-Python contributors.

# %%

import mne
import mne.datasets

import nilearn.datasets
import os
from pathlib import Path as Path

# The atlas is in a template space. We download here as an example Yeo 2011's atlas, which is in the MNI152 1mm template space.
# Replace this part with your atlas and the template space you used.

nilearn.datasets.fetch_atlas_yeo_2011() # Download Yeo 2011
yeo_path = Path(nilearn.datasets.get_data_dirs()[0], "yeo_2011", "Yeo_JNeurophysiol11_MNI152")
atlas_path = Path(yeo_path, "Yeo2011_7Networks_MNI152_FreeSurferConformed1mm.nii.gz")
atlas_template_T1_path = Path(yeo_path, "FSL_MNI152_FreeSurferConformed_1mm.nii.gz")

# The participant's T1 data. Here, we consider the sample dataset
# The brain should be skull stripped. After freesurfer preprocessing, you can either use brain.mgz or antsdn.brain.mgz
data_path = mne.datasets.sample.data_path()
subjects_mri_dir = Path(data_path, "subjects")
subject_mri_path = Path(subjects_mri_dir, "sample")
mri_path = Path(subject_mri_path, "mri")
T1_participant_path = Path(mri_path, "brain.mgz")

assert os.path.exists(atlas_path)
assert os.path.exists(atlas_template_T1_path)
assert os.path.exists(T1_participant_path)

#%%
# The first step is to put the atlas in subject space.
# We show this step with FSL and freesurfer with linear coregistration. If your atlas is already in participant space,
# you can skip this step. Coregistration is done in two steps: compute the atlas template to subject T1 transform and apply this transform
# to the atlas file with nearest neighbour interpolation.

# FSL does not know how to read .mgz, so we need to convert the T1 to nifti format
# With FreeSurfer:
T1_participant_nifti = str(T1_participant_path).replace("mgz", "nii.gz")
os.system("mri_convert {} {}".format(T1_participant_path, T1_participant_nifti))

# Compute template to subject anatomical transform
template_to_anat_transform_path = Path(mri_path, "template_to_anat.mat")
os.system("flirt -in {} -ref {} -out {} -omat {}".format(atlas_template_T1_path, T1_participant_nifti, Path(mri_path, "T1_atlas_coreg"), template_to_anat_transform_path))

# Apply the transform to the atlas
atlas_participant = Path(mri_path, "yeo_atlas.nii.gz")
os.system("flirt -in {} -ref {} -out {} -applyxfm -init {} -interp nearestneighbour".format(atlas_path, T1_participant_nifti, atlas_participant, template_to_anat_transform_path))

# Convert resulting atlas from nifti to mgz
# The filename must finish with aseg, to indicate to MNE that it is a proper atlas segmentation.
atlas_converted = str(atlas_participant).replace(".nii.gz", "aseg.mgz")
os.system("mri_convert {} {}".format(atlas_participant, atlas_converted))

assert os.path.exists(T1_participant_nifti)
assert os.path.exists(template_to_anat_transform_path)
assert os.path.exists(atlas_participant)
assert os.path.exists(atlas_converted)

#%%
# With the atlas in participant space, we're still missing one ingredient.
# We need a dictionary mapping label to region ID / value in the fMRI.
# In FreeSurfer and atlases, these typically take the form of lookup tables.
# You can also build the dictionary by hand.

from mne._freesurfer import read_freesurfer_lut
atlas_labels = read_freesurfer_lut(Path(yeo_path, "Yeo2011_7Networks_ColorLUT.txt"))[0]
print(atlas_labels)

# Drop the key corresponding to outer region
del atlas_labels["NONE"]

#%%
# For the purpose of source reconstruction, let's create a volumetric source estimate and source reconstruction with e.g eLORETA.
from mne.minimum_norm import apply_inverse, make_inverse_operator

vol_src = mne.setup_volume_source_space("sample", subjects_dir=subjects_mri_dir,
                                                surface=Path(subject_mri_path, "bem", "inner_skull.surf"))

fif_path = Path(data_path, "MEG", "sample")
fname_trans = Path(fif_path, "sample_audvis_raw-trans.fif")
raw_fname = Path(fif_path, "sample_audvis_filt-0-40_raw.fif")

model = mne.make_bem_model(subject="sample", subjects_dir=subjects_mri_dir, ico=4, conductivity=(0.33,))  
bem_sol = mne.make_bem_solution(model)

info = mne.io.read_info(raw_fname)
info = mne.pick_info(info, mne.pick_types(info, meg=True, eeg=False, exclude=[]))

# Build the forward model with our custom source
fwd = mne.make_forward_solution(info, trans=fname_trans, src=vol_src, bem=bem_sol)


# Now perform typical source reconstruction steps
raw = mne.io.read_raw_fif(raw_fname)  # already has an average reference
events = mne.find_events(raw, stim_channel="STI 014")

event_id = dict(aud_l=1)  # event trigger and conditions
tmin = -0.2  # start of each epoch (200ms before the trigger)
tmax = 0.5  # end of each epoch (500ms after the trigger)
raw.info["bads"] = ["MEG 2443", "EEG 053"]
baseline = (None, 0)  # means from the first instant to t = 0
reject = dict(grad=4000e-13, mag=4e-12, eog=150e-6)

epochs = mne.Epochs(
    raw,
    events,
    event_id,
    tmin,
    tmax,
    proj=True,
    picks=("meg", "eog"),
    baseline=baseline,
    reject=reject,
)

# Compute noise covariances
noise_cov = mne.compute_covariance(
    epochs, tmax=0.0, method=["shrunk", "empirical"], rank=None, verbose=True
)

# Compute evoked response
evoked = epochs.average().pick("meg")

# Make inverse operator
inverse_operator = make_inverse_operator(
    evoked.info, fwd, noise_cov, loose=1, depth=0.8
)

# Compute source time courses
method = "eLORETA"
snr = 3.0
lambda2 = 1.0 / snr**2
stc, residual = apply_inverse(
    evoked,
    inverse_operator,
    lambda2,
    method=method,
    pick_ori=None,
    return_residual=True,
    verbose=True,
)

#%%
# Then, we can finally use our atlas!
label_tcs = stc.extract_label_time_course(labels=(atlas_converted,atlas_labels),src=vol_src)
label_tcs.shape

mri_convert /home/guibert/mne_data/MNE-sample-data/subjects/sample/mri/brain.mgz /home/guibert/mne_data/MNE-sample-data/subjects/sample/mri/brain.nii.gz 
reading from /home/guibert/mne_data/MNE-sample-data/subjects/sample/mri/brain.mgz...
TR=2300.00, TE=1.64, TI=1100.00, flip angle=9.00
i_ras = (-1, 8.56817e-08, 1.49012e-08)
j_ras = (1.15484e-07, 1.57161e-08, -1)
k_ras = (-1.91852e-07, 1, 6.40284e-09)
writing to /home/guibert/mne_data/MNE-sample-data/subjects/sample/mri/brain.nii.gz...
mri_convert /home/guibert/mne_data/MNE-sample-data/subjects/sample/mri/yeo_atlas.nii.gz /home/guibert/mne_data/MNE-sample-data/subjects/sample/mri/yeo_atlasaseg.mgz 
reading from /home/guibert/mne_data/MNE-sample-data/subjects/sample/mri/yeo_atlas.nii.gz...
TR=1000.00, TE=0.00, TI=0.00, flip angle=0.00
i_ras = (-1, 8.56817e-08, 1.49012e-08)
j_ras = (1.15484e-07, 1.57161e-08, -1)
k_ras = (-1.91852e-07, 1, 6.40284e-09)
writing to /home/guibert/mne_data/MNE-sample-data/subjects/sample/mri/yeo_atlasaseg.mgz.

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


    Found   846/10797 points outside using solid angles
    Total 12738/22941 points inside the surface
Interior check completed in 5584.5 ms
    10203 source space points omitted because they are outside the inner skull surface.
    2362 source space points omitted because of the    5.0-mm distance limit.
10376 sources remaining after excluding the sources outside the surface and less than    5.0 mm inside.
Adjusting the neighborhood info.
Source space : MRI voxel -> MRI (surface RAS)
     0.005000  0.000000  0.000000     -70.00 mm
     0.000000  0.005000  0.000000     -90.00 mm
     0.000000  0.000000  0.005000     -45.00 mm
     0.000000  0.000000  0.000000       1.00
MRI volume : MRI voxel -> MRI (surface RAS)
    -0.001000  0.000000  0.000000     128.00 mm
     0.000000  0.000000  0.001000    -128.00 mm
     0.000000 -0.001000  0.000000     128.00 mm
     0.000000  0.000000  0.000000       1.00
MRI volume : MRI (surface RAS) -> RAS (non-zero origin)
     1.000000 -0.000000 -0.0000

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    5.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    5.6s finished


    8823320/16777216 nonzero values for the whole brain
[done]
Creating the BEM geometry...
Going from 4th to 4th subdivision of an icosahedron (n_tri: 5120 -> 5120)
inner skull CM is   0.67 -10.01  44.26 mm
Surfaces passed the basic topology checks.
Complete.

Homogeneous model surface loaded.
Computing the linear collocation solution...
    Matrix coefficients...
        inner skull (2562) -> inner skull (2562) ...
    Inverting the coefficient matrix...
Solution ready.
BEM geometry computations complete.
    Read a total of 4 projection items:
        PCA-v1 (1 x 102)  idle
        PCA-v2 (1 x 102)  idle
        PCA-v3 (1 x 102)  idle
        Average EEG reference (1 x 60)  idle
Source space          : <SourceSpaces: [<volume, shape=(29, 35, 32), n_used=10376>] MRI (surface RAS) coords, subject 'sample', ~72.0 MB>
MRI -> head transform : /home/guibert/mne_data/MNE-sample-data/MEG/sample/sample_audvis_raw-trans.fif
Measurement data      : instance of Info
Conductor model   : instance

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


    Found     0/ 7598 points outside using solid angles
    Total 10376/10376 points inside the surface
Interior check completed in 2980.5 ms

Checking surface interior status for 306 points...
    Found   0/306 points inside  an interior sphere of radius   43.6 mm
    Found 306/306 points outside an exterior sphere of radius   91.8 mm
    Found   0/  0 points outside using surface Qhull
    Found   0/  0 points outside using solid angles
    Total 0/306 points inside the surface
Interior check completed in 18.4 ms

Composing the field computation matrix...


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Computing MEG at 10376 source locations (free orientations)...


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
